In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import math 
import collections
from pycocotools.coco import COCO
import requests
import plotly.express as px
import plotly.graph_objects as go
from os import listdir
from os.path import isfile, join
import base64
import itertools
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
sns.set(style='whitegrid', font_scale=1.6, font='Georgia', context='paper')

from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import operator
import plotly.figure_factory as ff
import scipy
import pickle 
from sklearn.neighbors import KernelDensity
from tqdm import tqdm


/tmp/ipykernel_50178/3740038047.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
def max_confidence_multilabel(df, model, training_budget, dataset='animals'):
    targets = df['Target'].tolist()
    targets = [i[0] for i in targets]
    df['Target'] = targets
    orig_df = df.copy()
    all_ids = df['question_id'].to_list()

    # sampled_question_ids = []
    # sampled_variabilities = []
    # sampled_confidence = []
    # sampled_correctness = []
    # sampled_targets = []

    #unique_targets = df['Target'].unique()
    targets_list = np.array(df['Target'].tolist())
    question_ids = np.array(df['question_id'].to_list())
    confidence = np.array(df['confidence'].tolist())


    targets_multilabel =[] # unique targets in vqa dataset 
    for i in targets_list:
        target_list = [x.strip() for x in i.split(',')]
        targets_multilabel.extend(target_list)
    unique_targets = set(targets_multilabel)

    class_buckets = {}
    # class_lengths = {}
    for label in unique_targets:
        class_buckets[label] = {}
        # class_lengths[label] = 0
        
    
    
    for label in tqdm(unique_targets):
        for idx, target_name in enumerate(targets_list):
            if label in target_name:
                class_buckets[label][question_ids[idx]] = confidence[idx] # map the target to a question id and its associated confidence
# class_buckets now contains all the labels mapped to a dictionary that contains question_ids:confidence where question_id's target has that label

    # for label in unique_targets:
    #     class_lengths[label] = len(class_buckets[label])
    class_buckets_sorted_length = sorted(class_buckets, key=lambda k: len(class_buckets[k])) # sort classes from least to greatest
    
    class_buckets_sorted_confidence = {}
    for label in class_buckets_sorted_length:
        class_buckets_sorted_confidence[label] = []
    for label in tqdm(class_buckets_sorted_length):
        label_confidence_sorted = sorted(class_buckets_sorted_length[label].items(), key=lambda x:x[1]) # sort each class by confidence
        class_buckets_sorted_confidence[label].extend(list(label_confidence_sorted.keys()))
    

    final_sample = []

    for label in tqdm(class_buckets_sorted_confidence):
        num_samples = round(len(class_buckets_sorted_confidence[label]) * (training_budget*0.01))
        if num_samples == 0:
            num_samples = 1
        added_samples = 0
        label_idx = 0
        while added_samples <= num_samples and label_idx < len(class_buckets_sorted_confidence[label]):
            if class_buckets_sorted_confidence[label][label_idx] not in final_sample:
                final_sample.append(class_buckets_sorted_confidence[label][label_idx])
                added_samples +=1
            label_idx+=1

    save_path = 'src/dataset_selection/sampling/samples/'+model+'/'+dataset+'/max_confidence/seed_'+str(965)+'/budget_'+str(training_budget)+'.pkl'


    unique_targets_sample = orig_df[orig_df['question_id'].isin(final_sample)]
    sampled_targets_unique =[] # unique targets in vqa dataset 
    targets_list_sampled = np.array(unique_targets_sample['Target'].tolist())
    for i in targets_list_sampled:
        target_list = [x.strip() for x in i.split(',')]
        sampled_targets_unique.extend(target_list)
    unique_targets_sampled = set(sampled_targets_unique)

    # sampled_targets_unique = [] # unique targets in sampled data
    # sampled_targets = set(unique_targets_sample['Target'].unique())
    # for sample in sampled_targets:
    #     target_list = [x.strip() for x in sample.split(',')]
    #     sampled_targets_unique.extend(target_list)
    # sampled_targets_unique = set(sampled_targets_unique)
    targets_excluded = unique_targets - unique_targets_sampled
    print("TARGETS excluded: ", len(targets_excluded))

    
    with open(save_path, 'wb') as f:
        pickle.dump(list(set(sampled_question_ids)), f)

    print("unique targets max confidence per class: ", len(set(unique_targets_sampled)))
    print('samples - max confidence per class: ', len(set(final_sample)))
    print('all_samples - max confidence per class: ', len(all_ids))

In [8]:
!ls ../../../

config		  experience_in_pretraining.md	README.md	  src
data		  jupyterhub_cookie_secret	requirements.txt  test.py
datamaps	  jupyterhub.sqlite		run
environment.yaml  LICENSE			snap


In [3]:
base_path = '../../../snap/vqa/lxr111_multilabel_full_run_3/'
df = pd.read_pickle(base_path+"datamap_metrics.pkl")
sampling_dataset = 'multilabel_full'
sampling_model = 'LXR111'
max_confidence_multilabel(df, sampling_model, 30, dataset=sampling_dataset)

 23%|██▎       | 710/3128 [03:34<12:31,  3.22it/s]